In [3]:
import pandas as pd
import numpy as np
import sqlite3

import scipy as sp
import sklearn as skl

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import cross_validate,KFold
from sklearn import metrics
from sklearn import cross_validation
from sklearn import preprocessing
import csv
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import sklearn.model_selection as ms

In [4]:
engine = sqlite3.connect('CleanData-Apr15')

totaldata_df = pd.read_sql('select * from data', engine)
train_data, test_data= ms.train_test_split(totaldata_df, test_size=0.3, random_state=42)
scaler = StandardScaler()
scaler.fit(train_data[['CENTROID_X','CENTROID_Y','DistCBD','pop_density','housing_ratio']])
train_data[['CENTROID_X','CENTROID_Y','DistCBD','pop_density','housing_ratio']] =scaler.transform(train_data[['CENTROID_X','CENTROID_Y','DistCBD','pop_density','housing_ratio']])
test_data[['CENTROID_X','CENTROID_Y','DistCBD','pop_density','housing_ratio']] =scaler.transform(test_data[['CENTROID_X','CENTROID_Y','DistCBD','pop_density','housing_ratio']])

X_train=train_data.drop(labels=['WIDTH'],axis='columns')
y_train=train_data[['WIDTH']]
X_test=test_data.drop(labels=['WIDTH'],axis='columns')
y_test=test_data[['WIDTH']]


/Users/ilangold/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/ilangold/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/ilangold/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [5]:
print(totaldata_df.columns.values)

['CENTROID_X' 'CENTROID_Y' 'DistCBD' 'WIDTH' 'pop_density' 'housing_ratio'
 '0' '1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30' '31'
 '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43' '44' '45' '46'
 '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57' '58' '59' '60' '61'
 '62' '63' '64' '65' '66' '67' '68' '69' '70' '71' '72' '73' '74' '75' '76'
 '77' '78' '79' '80' '81' '82' '83' '84' '85' '86' '87' '88' '89' '90' '91'
 '92' '93' '94' '95' '96' '97' '98' '99' '100' '101' '102' '103' '104'
 '105' '106' '107' '108' '109' '110' '111' '112' '113' '114' '115' '116'
 '117' '118' '119' '120' '121' '122' '123' '124' '125' '126' '127' '128'
 '129' '130' '131' '132' '133' '134' '135' '136' '137' '138' '139' '140'
 '141' '142' '143' '144' '145' '146' '147' '148' '149' '150' '151' '152'
 '153' '154' '155' '156' '157' '158' '159' '160' '161' '162' '163' '164'
 '165' '166' '167' '168' '169' '1

In [4]:
def cross_val(input_data,test_data,input_labels,test_labels,model,model_type='other',verbose=0):
    test_labels=test_labels.ravel()
    input_labels=input_labels.ravel()
    err_matnew=np.empty((1,2))
    kf = KFold(n_splits=7,random_state=9, shuffle=True)
    seed = 9
    np.random.seed(seed)
    count=0
    for train_index, test_index in kf.split(input_data):
        X_cvtrain = input_data.iloc[train_index]
        X_cvtest = input_data.iloc[test_index]

        y_cvtrain = input_labels[train_index]
        y_cvtrain=y_cvtrain.squeeze()
        y_cvtrain=y_cvtrain.ravel()

        y_cvtest = input_labels[test_index]
        y_cvtest=y_cvtest.squeeze()
        y_cvtest=y_cvtest.ravel()

        if model_type=='keras':
            model=keras_nn_model(X_cvtrain.shape[1],'relu',70)
            model.fit(X_cvtrain,y_cvtrain,epochs=10, batch_size=30, verbose=0)
        
        else:
            model.fit(X_cvtrain,y_cvtrain)
        predictions = model.predict(X_cvtest).ravel()

        err_vec=np.column_stack((y_cvtest,predictions))
        err_matnew=np.concatenate((err_matnew,err_vec),axis=0)
        temp1=np.square(predictions-y_cvtest)
        temp2=np.abs(predictions-y_cvtest)
        temp3=np.divide(temp2,y_cvtest)
        count+=1
        if verbose==1:
            print('cv error: '+str(count)+' Fold')
            print(np.array([np.sqrt(sum(temp1)/temp1.shape[0]),sum(temp2)/temp1.shape[0],100*sum(temp3)/temp1.shape[0]]))

    err_matnew=err_matnew[1:,:]     
    
    temp1=np.square(err_matnew[:,1]-err_matnew[:,0])
    temp2=np.abs(err_matnew[:,1]-err_matnew[:,0])
    temp3=np.divide(temp2,err_matnew[:,0])
    print('cv_error:')
    print(np.array([np.sqrt(sum(temp1)/temp1.shape[0]),sum(temp2)/temp1.shape[0],100*sum(temp3)/temp1.shape[0]]))
    
    
    
    if model_type=='keras':
        model=keras_nn_model(input_data.shape[1],'relu',70)
        model.fit(input_data,input_labels,epochs=10, batch_size=30, verbose=0)
    else:
        model.fit(input_data,input_labels)

    test_predict = model.predict(test_data).ravel()    
    temp1=np.square(test_predict-test_labels)
    temp2=np.abs(test_predict-test_labels)
    temp3=np.divide(temp2,test_labels)
    print('\n')
    print('test error: ')
    print(np.array([np.sqrt(sum(temp1)/temp1.shape[0]),sum(temp2)/temp1.shape[0],100*sum(temp3)/temp1.shape[0]]))
    print('\n')
    return err_matnew,np.column_stack((test_labels,test_predict)),model

In [5]:
#cross validation to pick the best Neural Net Model

#for transforming into interpretable Errors
#scaler = StandardScaler()
#scaler.fit(fdata['WIDTH'].values.reshape(-1, 1))


reg=[0.00001, .0001, .001, .01, .1, 0, 1]
layer_size1=[50, 70, 100]
out_layer = 0
out_reg = 0
err_min = np.inf
err_perc = np.inf
err_abs = np.inf
for l1 in layer_size1:
    for r in reg:   
        for act in ['identity', 'logistic', 'tanh', 'relu']:
            layer=l1
            alph=r
            mlp = MLPRegressor(hidden_layer_sizes=layer,activation=act, max_iter = 100,alpha=alph, early_stopping = False)
            err_mat, testers ,model = cross_val(X_train, X_test, np.array(y_train), np.array(y_test), model = mlp,verbose=1)
            err_mat=err_mat[1:,:]     
            temp1=np.square(err_mat[:,1]-err_mat[:,0])
            err = np.sqrt(sum(temp1)/temp1.shape[0])
            if(err < err_min):
                err_min = err
                test_predict = model.predict(X_test).ravel()    
                temp1=np.square(test_predict-np.array(y_test).ravel())
                temp2=np.abs(test_predict-np.array(y_test).ravel())
                temp3=np.divide(temp2,np.array(y_test).ravel())
                err_sq, err_m, err_p = np.array([np.sqrt(sum(temp1)/temp1.shape[0]),sum(temp2)/temp1.shape[0],100*sum(temp3)/temp1.shape[0]])
                print('the new minimum error parameters are:')
                print('layers:')
                print(layer)
                print('reg parameter:')
                print(alph)
                print('the activation is:')
                print(act)
                print('the new minimum cv sq error is:')
                print(err)
                print('the new minimum test sq error is:')
                print(err_sq)
                print('the new minimum test percent error is:')
                print(err_p)
                print('the new minimum test percent error is:')
                print(err_m)
                print('\n')

cv error: 1 Fold
[  3.70976562   2.46595937  35.17287718]
cv error: 2 Fold
[  3.87189084   2.46006181  34.29288239]
cv error: 3 Fold
[  3.56357035   2.42776258  34.10453401]
cv error: 4 Fold
[  3.99165292   2.43379646  35.18651809]
cv error: 5 Fold
[  3.91997923   2.46979453  34.6850707 ]
cv error: 6 Fold
[  3.50006066   2.39503057  35.12980666]
cv error: 7 Fold
[  3.58220546   2.41668505  32.77495221]
cv_error:
[  3.73852665   2.43844468  34.47814323]


test error: 
[  3.92964019   2.44625979  33.82886752]


the new minimum error parameters are:
layers:
50
reg parameter:
1e-05
the activation is:
identity
the new minimum cv sq error is:
3.73861008908
the new minimum test sq error is:
3.92964018852
the new minimum test percent error is:
33.8288675187
the new minimum test percent error is:
2.44625978525




/Users/ilangold/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


cv error: 1 Fold
[  3.66960862   2.42387579  34.33551625]
cv error: 2 Fold
[  3.82805882   2.40717222  33.5123789 ]
cv error: 3 Fold
[  3.50825396   2.34634255  32.33965897]
cv error: 4 Fold
[  3.94216937   2.37228095  33.80283942]
cv error: 5 Fold
[  3.88185215   2.44328532  34.83396921]
cv error: 6 Fold
[  3.44269095   2.33017461  33.65041391]
cv error: 7 Fold
[  3.52533686   2.38415359  32.95289861]
cv_error:
[  3.69011264   2.38675791  33.63255753]


test error: 
[  3.87572712   2.40501474  33.50329241]


the new minimum error parameters are:
layers:
50
reg parameter:
1e-05
the activation is:
logistic
the new minimum cv sq error is:
3.69019827661
the new minimum test sq error is:
3.87572712348
the new minimum test percent error is:
33.5032924115
the new minimum test percent error is:
2.40501474189


cv error: 1 Fold
[  3.66878829   2.41507879  33.78763205]
cv error: 2 Fold
[  3.80499921   2.35988689  32.35302809]
cv error: 3 Fold
[  3.51441545   2.33557975  31.68375086]
cv error: 4

cv error: 7 Fold
[  3.50802813   2.36637202  32.86892681]
cv_error:
[  3.68113302   2.37369285  33.25072915]


test error: 
[  3.86320926   2.4032366   33.66064986]


the new minimum error parameters are:
layers:
50
reg parameter:
0.01
the activation is:
tanh
the new minimum cv sq error is:
3.6812166855
the new minimum test sq error is:
3.86320925826
the new minimum test percent error is:
33.6606498596
the new minimum test percent error is:
2.40323660206


cv error: 1 Fold
[  3.74675094   2.46885911  34.17759911]
cv error: 2 Fold
[  3.87322475   2.41231193  32.97703479]
cv error: 3 Fold
[  3.60520954   2.39724789  32.00462606]
cv error: 4 Fold
[  4.01689275   2.42642624  33.62375875]
cv error: 5 Fold
[  3.92877746   2.4719467   34.94488331]
cv error: 6 Fold
[  3.5429421    2.38376988  33.83900848]
cv error: 7 Fold
[  3.5593624    2.40562534  33.01586371]
cv_error:
[  3.75748403   2.42374386  33.51183317]


test error: 
[  3.9204368    2.44343953  33.86690384]


cv error: 1 Fold
[  3.70



test error: 
[  3.92659613   2.46102933  34.56498318]


cv error: 1 Fold
[  3.66985016   2.42140002  34.25604496]
cv error: 2 Fold
[  3.82625728   2.41393275  33.82547529]
cv error: 3 Fold
[  3.50890277   2.35155761  32.5737638 ]
cv error: 4 Fold
[  3.94272682   2.37372824  33.816174  ]
cv error: 5 Fold
[  3.87999887   2.41727483  33.91657881]
cv error: 6 Fold
[  3.44402959   2.32469783  33.4054078 ]
cv error: 7 Fold
[  3.52570817   2.37588893  32.60280022]
cv_error:
[  3.69000383   2.3826432   33.48522508]


test error: 
[  3.87495647   2.40298721  33.45324134]


cv error: 1 Fold
[  3.67254545   2.41775378  33.93387651]
cv error: 2 Fold
[  3.81595576   2.41343169  34.11336012]
cv error: 3 Fold
[  3.51598767   2.3532672   32.52299281]
cv error: 4 Fold
[  3.94498      2.37443103  33.69608515]
cv error: 5 Fold
[  3.88446875   2.42780247  34.11695003]
cv error: 6 Fold
[  3.41556761   2.31297542  33.07599822]
cv error: 7 Fold
[  3.50825491   2.35279953  32.40043955]
cv_error:
[  3.684684



test error: 
[  3.92653892   2.44334759  33.82579373]


cv error: 1 Fold
[  3.6845399    2.42663611  34.44720271]
cv error: 2 Fold
[  3.84978523   2.44544464  34.54684832]
cv error: 3 Fold
[  3.52079737   2.36277267  32.83452298]
cv error: 4 Fold
[  3.95193547   2.38348746  34.23198427]
cv error: 5 Fold
[  3.87995972   2.45188351  35.00792202]
cv error: 6 Fold
[  3.46169916   2.35997603  34.39150241]
cv error: 7 Fold
[  3.53999667   2.39471645  33.14846093]
cv_error:
[  3.70290451   2.40356213  34.08695164]


test error: 
[  3.8917706    2.41153164  33.68434912]


cv error: 1 Fold
[  3.65855082   2.40883287  33.9237835 ]
cv error: 2 Fold
[  3.80750412   2.41216672  34.20885667]
cv error: 3 Fold
[  3.50639528   2.3489635   32.59074203]
cv error: 4 Fold
[  3.93201032   2.36717123  33.70757163]
cv error: 5 Fold
[  3.87319636   2.42081446  34.07941912]
cv error: 6 Fold
[  3.40460664   2.30471126  33.13321216]
cv error: 7 Fold
[  3.50327751   2.35229232  32.4457002 ]
cv_error:
[  3.674324

cv error: 4 Fold
[  3.99049239   2.43618574  35.21971182]
cv error: 5 Fold
[  3.92544948   2.45817918  34.2445544 ]
cv error: 6 Fold
[  3.49463095   2.36265385  34.02008078]
cv error: 7 Fold
[  3.58036655   2.42700014  33.22357662]
cv_error:
[  3.73698371   2.43095556  34.24881113]


test error: 
[  3.93015276   2.44630182  34.01353542]


cv error: 1 Fold
[  3.67291238   2.4206253   34.1617796 ]
cv error: 2 Fold
[  3.82923137   2.40512776  33.42651017]
cv error: 3 Fold
[  3.51557792   2.32876329  31.49721246]
cv error: 4 Fold
[  3.94487687   2.38467636  34.15774085]
cv error: 5 Fold
[  3.88112758   2.42551787  34.21728855]
cv error: 6 Fold
[  3.44421887   2.32980023  33.52064628]
cv error: 7 Fold
[  3.53035056   2.36118969  31.96764989]
cv_error:
[  3.69294353   2.37938911  33.27845643]


test error: 
[  3.87660775   2.43756719  34.60824525]


cv error: 1 Fold
[  3.66477315   2.40721053  33.64750885]
cv error: 2 Fold
[  3.80884804   2.36778804  32.71379372]
cv error: 3 Fold
[  3.520954

cv error: 4 Fold
[  3.99049452   2.43619242  35.21974078]
cv error: 5 Fold
[  3.92549456   2.45822314  34.24485618]
cv error: 6 Fold
[  3.49463759   2.36266566  34.02023326]
cv error: 7 Fold
[  3.58036573   2.42697395  33.22329554]
cv_error:
[  3.73698978   2.43095993  34.24881779]


test error: 
[  3.93015616   2.44630794  34.01367154]


cv error: 1 Fold
[  3.67296969   2.42066284  34.16190761]
cv error: 2 Fold
[  3.82924372   2.40500955  33.42377161]
cv error: 3 Fold
[  3.51539088   2.32856327  31.49238511]
cv error: 4 Fold
[  3.94490405   2.38468511  34.15703708]
cv error: 5 Fold
[  3.88124371   2.42560223  34.21780692]
cv error: 6 Fold
[  3.44427426   2.32982921  33.52061274]
cv error: 7 Fold
[  3.53042622   2.36120502  31.96688622]
cv_error:
[  3.69296736   2.37936864  33.27725342]


test error: 
[  3.87668831   2.43763608  34.60863561]


cv error: 1 Fold
[  3.66471375   2.40713704  33.64702209]
cv error: 2 Fold
[  3.80892694   2.3678522   32.71448097]
cv error: 3 Fold
[  3.521025

In [20]:
#best is 100 hidden units, 1 regularization
mlp = MLPRegressor(hidden_layer_sizes=100,activation='relu', max_iter=1000, alpha=1, early_stopping=True, random_state=42)
mlp.fit(X_train, np.array(y_train).ravel())
test_predict = mlp.predict(X_test).ravel()    
temp1=np.square(test_predict-np.array(y_test).ravel())
temp2=np.abs(test_predict-np.array(y_test).ravel())
temp3=np.divide(temp2,np.array(y_test).ravel())
err = np.array([np.sqrt(sum(temp1)/temp1.shape[0]),sum(temp2)/temp1.shape[0],100*sum(temp3)/temp1.shape[0]])
print(err[0], err[1], err[2])

3.89683149321 2.42505560131 33.9822775856


In [21]:
#next we fit the forest and choose sample_splits by CV
min_samples_splits = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 38, 40, 42, 44, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68]
err_min = np.inf
err_perc = np.inf
err_abs = np.inf
out_split = 0
y_train=y_train.squeeze()
y_train=y_train.ravel()
y_test=y_test.squeeze()
y_test=y_test.ravel()
for splits in min_samples_splits:
    kf = KFold(n_splits=8,random_state=9, shuffle=True)
    err = np.zeros(8)
    err_p = np.zeros(8)
    err_m = np.zeros(8)
    i=0
    for train_index, test_index in kf.split(X_train):
        X_cvtrain = X_train.iloc[train_index]
        X_cvtest = X_train.iloc[test_index]
        y_cvtrain = y_train[train_index]
        y_cvtest = y_train[test_index]
        regr = RandomForestRegressor(n_estimators = 30, min_samples_split = splits, random_state=42)
        regr.fit(X_cvtrain,y_cvtrain)
        predictions = regr.predict(X_cvtest)
        err[i] = np.sqrt(np.sum(np.square(predictions - y_cvtest)) / len(predictions))
        err_p[i] = np.sum(np.divide(np.abs(predictions - y_cvtest), y_cvtest) / len(predictions))
        err_m[i] = np.sum(np.abs(predictions - y_cvtest) / len(predictions))
        i=i+1
    err = np.mean(err)
    err_p = np.mean(err_p)
    err_m = np.mean(err_m)
    if(err < err_min):
        err_min = err
        print('the new minimum error parameters are:')
        print('splits:')
        print(splits)
        print('the sq cv error is:')
        print(err)
        print('the sq test error is:')
        predictions = regr.predict(X_test)
        print(np.sqrt(sum(np.square(predictions - y_test)) / len(predictions)))
        print('\n') 
    if(err_p < err_perc):
        err_perc = err_p
        print('the new minimum error parameters are:')
        print('splits:')
        print(splits)
        print('the perc cv error is:')
        print(err_p)
        print('the perc test error is:')
        predictions = regr.predict(X_test)
        print(sum(np.divide(np.abs(predictions - y_test), y_test) / len(predictions)))
        print('\n') 
    if(err_m < err_abs):
        err_abs = err_m
        print('the new minimum error parameters are:')
        print('splits:')
        print(splits)
        print('the mae cv error is:')
        print(err_m)
        print('the mae test error is:')
        predictions = regr.predict(X_test)
        print(np.sum(np.abs(predictions - y_test) / len(predictions)))
        print('\n') 

the new minimum error parameters are:
splits:
2
the sq cv error is:
3.88649502366
the sq test error is:
3.99822880884


the new minimum error parameters are:
splits:
2
the perc cv error is:
0.347563042094
the perc test error is:
0.34559204367


the new minimum error parameters are:
splits:
2
the mae cv error is:
2.49407685115
the mae test error is:
2.50181543948


the new minimum error parameters are:
splits:
4
the sq cv error is:
3.86000495473
the sq test error is:
3.98195986756


the new minimum error parameters are:
splits:
4
the perc cv error is:
0.346499398347
the perc test error is:
0.344855482465


the new minimum error parameters are:
splits:
4
the mae cv error is:
2.48200722399
the mae test error is:
2.49109954705


the new minimum error parameters are:
splits:
6
the sq cv error is:
3.83993095168
the sq test error is:
3.96347310589


the new minimum error parameters are:
splits:
6
the perc cv error is:
0.344954090202
the perc test error is:
0.342942115979


the new minimum err

the new minimum error parameters are:
splits:
54
the sq cv error is:
3.70892614882
the sq test error is:
3.87933689773


the new minimum error parameters are:
splits:
54
the perc cv error is:
0.333310752277
the perc test error is:
0.334212966236


the new minimum error parameters are:
splits:
54
the mae cv error is:
2.38171221835
the mae test error is:
2.39847279546


the new minimum error parameters are:
splits:
56
the sq cv error is:
3.7082079125
the sq test error is:
3.87996890489


the new minimum error parameters are:
splits:
56
the perc cv error is:
0.33327785534
the perc test error is:
0.334220325603


the new minimum error parameters are:
splits:
56
the mae cv error is:
2.38106135539
the mae test error is:
2.39856341825


the new minimum error parameters are:
splits:
58
the sq cv error is:
3.70695068094
the sq test error is:
3.88059344232


the new minimum error parameters are:
splits:
58
the perc cv error is:
0.333202530149
the perc test error is:
0.334056469423


the new mini

In [7]:
#fit chosen RF model
regr = RandomForestRegressor(n_estimators = 30, min_samples_split = 68, random_state=0)
regr.fit(X_train,y_train)
predictions = regr.predict(X_test)

/Users/ilangold/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [8]:
#get feautre importance
vals = pd.concat([pd.DataFrame(X_train.columns), pd.DataFrame(regr.feature_importances_)], axis = 1)
vals.columns = ['features', 'importance']
print(vals.sort_values(by = ['importance']))

          features  importance
131            126    0.000000
27              22    0.000000
51              46    0.000000
57              52    0.000000
321            316    0.000000
75              70    0.000000
84              79    0.000000
102             97    0.000000
109            104    0.000000
115            110    0.000000
16              11    0.000000
294            289    0.000000
34              29    0.000000
33              28    0.000000
158            153    0.000000
269            264    0.000000
200            195    0.000000
219            214    0.000000
232            227    0.000000
234            229    0.000000
239            234    0.000000
145            140    0.000000
175            170    0.000002
228            223    0.000004
274            269    0.000004
325            320    0.000005
153            148    0.000006
243            238    0.000007
250            245    0.000009
305            300    0.000010
..             ...         ...
18      

In [13]:
#next we fit the tree and choose sample_splits by CV
min_samples_splits = [2, 6, 10, 14, 18, 22, 26, 30, 34, 38, 42, 46, 50, 54, 58, 62, 66, 70, 74, 78, 82]
err_min = np.inf
err_perc = np.inf
err_abs = np.inf
out_split = 0
y_train=y_train.squeeze()
y_train=y_train.ravel()
y_test=y_test.squeeze()
y_test=y_test.ravel()
for splits in min_samples_splits:
    for depth in [5, 8, 10, None]:
        kf = KFold(n_splits=8,random_state=9, shuffle=True)
        err = np.zeros(8)
        err_p = np.zeros(8)
        err_m = np.zeros(8)
        i=0
        for train_index, test_index in kf.split(X_train):
            X_cvtrain = X_train.iloc[train_index]
            X_cvtest = X_train.iloc[test_index]
            y_cvtrain = y_train[train_index]
            y_cvtest = y_train[test_index]
            regr = DecisionTreeRegressor(min_samples_split = splits, max_depth=depth, random_state=314)
            regr.fit(X_cvtrain,y_cvtrain)
            predictions = regr.predict(X_cvtest)
            err[i] = np.sqrt(np.sum(np.square(predictions - y_cvtest)) / len(predictions))
            err_p[i] = np.sum(np.divide(np.abs(predictions - y_cvtest), y_cvtest) / len(predictions))
            err_m[i] = np.sum(np.abs(predictions - y_cvtest) / len(predictions))
            i=i+1
        err = np.mean(err)
        err_p = np.mean(err_p)
        err_m = np.mean(err_m)
        if(err < err_min):
            err_min = err
            print('the new minimum error parameters are:')
            print('splits:')
            print(splits)
            print('depth:')
            print(depth)
            print('the sq cv error is:')
            print(err)
            print('the sq test error is:')
            predictions = regr.predict(X_test)
            print(np.sqrt(sum(np.square(predictions - y_test)) / len(predictions)))
            print('\n') 
        if(err_p < err_perc):
            err_perc = err_p
            print('the new minimum error parameters are:')
            print('splits:')
            print(splits)
            print('depth:')
            print(depth)
            print('the perc cv error is:')
            print(err_p)
            print('the perc test error is:')
            predictions = regr.predict(X_test)
            print(sum(np.divide(np.abs(predictions - y_test), y_test) / len(predictions)))
            print('\n') 
        if(err_m < err_abs):
            err_abs = err_m
            print('the new minimum error parameters are:')
            print('splits:')
            print(splits)
            print('depth:')
            print(depth)
            print('the mae cv error is:')
            print(err_m)
            print('the mae test error is:')
            predictions = regr.predict(X_test)
            print(np.sum(np.abs(predictions - y_test) / len(predictions)))
            print('\n') 

the new minimum error parameters are:
splits:
2
depth:
5
the sq cv error is:
3.8407807079
the sq test error is:
3.96274166468


the new minimum error parameters are:
splits:
2
depth:
5
the perc cv error is:
0.347823850241
the perc test error is:
0.3474334739


the new minimum error parameters are:
splits:
2
depth:
5
the mae cv error is:
2.45486071038
the mae test error is:
2.47320878457


the new minimum error parameters are:
splits:
2
depth:
8
the perc cv error is:
0.344201379806
the perc test error is:
0.342095515193


the new minimum error parameters are:
splits:
6
depth:
5
the sq cv error is:
3.83225391305
the sq test error is:
3.95892774159


the new minimum error parameters are:
splits:
6
depth:
5
the mae cv error is:
2.45428278618
the mae test error is:
2.47159656615


the new minimum error parameters are:
splits:
10
depth:
8
the perc cv error is:
0.343706610075
the perc test error is:
0.340206972484


the new minimum error parameters are:
splits:
14
depth:
8
the perc cv error i

In [17]:
#boost the tree
regr = DecisionTreeRegressor(min_samples_split = 82, max_depth = 5, random_state=314)
err_min = np.inf
err_perc = np.inf
err_abs = np.inf
for learning_rt in [.00001, .0001, .001, .01, .1]:
    for ls in ['square', 'linear', 'exponential']:
        boost = AdaBoostRegressor(base_estimator=regr, learning_rate=learning_rt, random_state=314, loss=ls)
        boost.fit(X_train, y_train)
        predictions = boost.predict(X_test)
        err = np.sqrt(sum(np.square(predictions - y_test)) / len(predictions))
        err_p = np.sum(np.divide(np.abs(predictions - y_test), y_test) / len(predictions))
        err_m = np.sum(np.abs(predictions - y_test) / len(predictions))
        if(err < err_min):
            err_min = err
            print(ls)
            print(learning_rt)
            print('sq')
            print(err)
            print('\ln')
        if(err_p < err_perc):
            err_perc = err_p
            print(ls)
            print(learning_rt)
            print('perc')
            print(err_p)
            print('\ln')
        if(err_m < err_abs):
            err_abs = err_m
            print(ls)
            print(learning_rt)
            print('mae')
            print(err_p)
            print('\ln')

square
1e-05
sq
3.94543105388
\ln
square
1e-05
perc
0.343069673414
\ln
square
1e-05
mae
0.343069673414
\ln
linear
0.001
sq
3.94406044323
\ln
exponential
0.001
perc
0.342939494189
\ln
exponential
0.001
mae
0.342939494189
\ln
exponential
0.01
sq
3.94380975128
\ln
